In [10]:
# import dependencies
import pandas as pd
import json
import requests
from googleapiclient.discovery import build
from config import api_key

In [11]:
# Enter video id

# Snarky Puppy - Trinity
video_id = "CSiLPkUopBg"



In [27]:
def video_comments(video_id):
    # empty list for storing reply
    replies = []
  
    # creating youtube resource object
    youtube = build('youtube', 'v3', developerKey=api_key)
  
    # retrieve youtube video results
    video_response = youtube.commentThreads().list(part='snippet,replies', videoId=video_id).execute()
  
    # iterate video response
    while video_response:
        
        # extracting required info from each result object 
        for item in video_response['items']:
            
            # Extracting comments
            comment = item['snippet']['topLevelComment']['snippet']['textOriginal']
              
            # counting number of reply of comment
            replycount = item['snippet']['totalReplyCount']
  
            # if reply is there
            if replycount>0:
                
                # iterate through all reply
                for reply in item['replies']['comments']:
                    
                    # Extract reply
                    reply = reply['snippet']['textOriginal']
                      
                    # Store reply in list
                    replies.append(reply)
  
            # print comment with list of reply to text file
            with open('comments.txt', 'a', encoding="utf-8") as f:
                f.write(comment + str(replies) + '\n\n')

            print(comment, replies, end = '\n\n')
  
            # empty reply list
            replies = []
  
        # Again repeat on next page
        if 'nextPageToken' in video_response:
            video_response = youtube.commentThreads().list(
            part='snippet,replies',
            videoId=video_id,
            pageToken=video_response['nextPageToken']
            ).execute()
        else:
            break


In [28]:
video_comments(video_id)

Crazy how well the two drum sets play together/don't clash. []

I fell like God probably has Snarky Puppy on his play list... Such talent can only come from the creator.
Love listening to their music! []

https://www.youtube.com/shorts/8noX0Fu36fI []

Are all 16 tracks visually registered like this track? I only find a handful of em

Thx! ["They're releasing them slowly one by one"]

Is that Jack Conte I see at the 8:22 and 10:48 mark????? :) []

Best song on the album! ['Keep It On Your Mind?']

wooowwww... amazing i love you guys []

Fantastic []

I've been playing this song for at least twice a day for the last 3 weeks everyday .... This song touches my soul! Thank you ... []

Brilliant! Excited to see hopefully the most of you in Munich on Monday. []

East River La Di Dah! []

What a solo from Chris Bullock, i'm amazed []

Saw this track played live in Amsterdam this week. Absolutely blew me away. Great performance and energy all through the set []

I can only imagine what a giant 